In [1]:
import os 
import json
import glob 
import pandas as pd
from datetime import datetime, timezone, date
import requests
from IPython.display import clear_output
import random 
import time 
import numpy as np
import glob 

In [2]:
RAW_MATCH_DETAILS_DIR = "../Data/Raw/Match_details"

DNF_KEYWORDS = ['WO', 'INJ', 'RET', 'DSQ']
DNF_PATTERN = '|'.join(DNF_KEYWORDS)
DNF_PATTERN_CAPTURE = r'(' + r'|'.join(DNF_KEYWORDS) + r')'

drop_columns = [""

]

In [3]:
all_match_details_files = glob.glob(os.path.join(RAW_MATCH_DETAILS_DIR, "*.json"))
all_matches = []
for file in all_match_details_files:
    with open(file, 'r', encoding='utf-8') as f:
        matches_data = json.load(f)

    all_matches.extend(matches_data)







In [4]:
all_matches_df = pd.DataFrame(all_matches)
age_limit_mask = all_matches_df['subEventName'].str.contains(r"U\d{2}", case=False, na=False)
para_class_mask = all_matches_df['subEventName'].str.contains("class", case=False, na=False)
my_filter = age_limit_mask | para_class_mask

# a df without any para or age limited matches (i.e all in same event / class except for gender split)
cleaned_matches_df = all_matches_df[~my_filter].copy()
cleaned_matches_df["dnf"] = cleaned_matches_df["overallScores"].str.extract(DNF_PATTERN_CAPTURE, expand=False).str.strip()
#rejected_matches_df = all_matches_df[my_filter].copy()

In [5]:
# 1: Add dnf column. ads dnf reason or sets to FALSE    
cleaned_matches_df["dnf"] = cleaned_matches_df["overallScores"].str.extract(DNF_PATTERN_CAPTURE, expand=False).str.strip()



In [6]:
def extract_player_details_raw(competitor_list):
    """
    Extracts name and ID details for Home (H) and Away (A) competitors
    by pulling the raw data directly from the 'competitiors' column.
    
    NO parsing or splitting of names is performed.
    """
    
    data = {
        'home_player_id': pd.NA,
        'home_player_name': pd.NA,
        'home_given_name': pd.NA,
        'home_family_name': pd.NA,
        'away_player_id': pd.NA,
        'away_player_name': pd.NA,
        'away_given_name': pd.NA,
        'away_family_name': pd.NA,
    }
    
    # Safety check for empty or malformed data
    if not isinstance(competitor_list, list) or len(competitor_list) < 2:
        return pd.Series(data)

    try:
        for comp in competitor_list:
            comp_type = comp.get('competitorType')
            
            if comp_type == 'H':
                prefix = 'home_'
            elif comp_type == 'A':
                prefix = 'away_'
            else:
                continue
                
            # Safely get the (first) player dict, or an empty dict
            player_info = comp.get('players', [{}])[0]
            
            # --- Direct Extraction (No Parsing) ---
            
            # 1. Get ID (fallback from player-level to competitor-level)
            data[f'{prefix}player_id'] = player_info.get('playerId', comp.get('competitiorId', pd.NA))
            
            # 2. Get Player Name (fallback from player-level to competitor-level)
            data[f'{prefix}player_name'] = player_info.get('playerName', comp.get('competitiorName', pd.NA))
            
            # 3. Get Given Name (raw)
            data[f'{prefix}given_name'] = player_info.get('playerGivenName', pd.NA)
            
            # 4. Get Family Name (raw)
            data[f'{prefix}family_name'] = player_info.get('playerFamilyName', pd.NA)

    except Exception as e:
        print(f"Error processing row: {e} | Data: {competitor_list}")
        pass

    return pd.Series(data)

In [33]:
print("Processing Event:      ... (Flattening competitor data - raw)")

# 1. Apply the function to the 'competitiors' column
player_details_df = cleaned_matches_df['competitiors'].apply(extract_player_details_raw)

# 2. Join the new columns back to the main DataFrame
cleaned_matches_df = pd.concat(
    [cleaned_matches_df.drop(player_details_df.columns, axis=1, errors='ignore'), 
     player_details_df], 
    axis=1
)

print("✅ Competitor details extracted and appended (as is).")


Processing Event:      ... (Flattening competitor data - raw)
✅ Competitor details extracted and appended (as is).


In [35]:
# remove any matches where the away player name contains a number
home_contains_digit = cleaned_matches_df["home_player_name"].str.contains(r"[0-9]", na=False)
away_contains_digit = cleaned_matches_df["away_player_name"].str.contains(r"[0-9]", na=False)
keep_filter = ~(home_contains_digit | away_contains_digit)
cleaned_matches_df = cleaned_matches_df[keep_filter].copy()

In [52]:
print(f"before: {len(cleaned_matches_df)}")
away_given_name_contains_digit = cleaned_matches_df["away_given_name"].str.contains(r"[0-9]", na=False)
away_family_name_contains_digit = cleaned_matches_df["away_family_name"].str.contains(r"[0-9]", na=False)
home_given_name_contains_digit = cleaned_matches_df["home_given_name"].str.contains(r"[0-9]", na=False)
home_family_name_contains_digit = cleaned_matches_df["home_family_name"].str.contains(r"[0-9]", na=False)
keep_filter = ~(away_given_name_contains_digit | away_family_name_contains_digit | home_given_name_contains_digit | home_family_name_contains_digit)
cleaned_matches_df = cleaned_matches_df[keep_filter].copy()
print(f"after: {len(cleaned_matches_df)}")


before: 23547
after: 23547


In [53]:
print(f"before: {len(cleaned_matches_df)}")
home_name_contains_team = cleaned_matches_df["home_player_name"].str.contains("team", case=False, na=False)
away_name_contains_team = cleaned_matches_df["away_player_name"].str.contains("team", case=False, na=False)
home_given_name_contains_team = cleaned_matches_df["home_given_name"].str.contains("team", case=False, na=False)
away_given_name_contains_team = cleaned_matches_df["away_given_name"].str.contains("team", case=False, na=False)
home_family_name_contains_team = cleaned_matches_df["home_family_name"].str.contains("team", case=False, na=False)
away_family_name_contains_team = cleaned_matches_df["away_family_name"].str.contains("team", case=False, na=False)
keep_filter = ~(home_name_contains_team | away_name_contains_team | home_given_name_contains_team | away_given_name_contains_team | home_family_name_contains_team | away_family_name_contains_team)
cleaned_matches_df = cleaned_matches_df[keep_filter].copy()
print(f"after: {len(cleaned_matches_df)}")


before: 23547
after: 23547
